<a href="https://colab.research.google.com/github/sonmiannkim/Amazon_Vine_Analysis/blob/main/Amazon_Reviews_ETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:6 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu b

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-06-19 13:33:05--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  5.68MB/s    in 0.2s    

2022-06-19 13:33:06 (5.68 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Digital_Ebook_Purchase_v1_01.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   33605939| RGYFDX8QXKEIR|B007KO2MLO|     328837464|           Big Maria|Digital_Ebook_Pur...|          4|            0|          0|   N|                N|              Quirky|Elmore Leonard me...| 2013-09-09|
|         US|   34058393|R13CBGTMNV9R8Z|B005FLODDE|     764276359|The Woman Who Was...|Digital_Ebook_Pur

In [5]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame - review_id, customer_id,product_id, product_parent INTEGER, review_date DATE (yyyy-mm-dd)
raw_review_id_df = df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
raw_review_id_df.show(5)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| RGYFDX8QXKEIR|   33605939|B007KO2MLO|     328837464| 2013-09-09|
|R13CBGTMNV9R8Z|   34058393|B005FLODDE|     764276359| 2013-09-09|
| R7DRFHC0F71O0|   39601147|B00EA3L35O|     535606445| 2013-09-09|
|R27LUKEXU3KBXQ|   17351407|B00BL3JV50|     240053004| 2013-09-09|
|R1VXTPUYMNU687|   10463387|B00CXU7U80|     931529805| 2013-09-09|
+--------------+-----------+----------+--------------+-----------+
only showing top 5 rows



In [21]:
# Create the customers_table DataFrame - customer_id, customer_count
customers_df = df.groupby("customer_id").agg({"customer_id":"count"}).withColumnRenamed("count(customer_id)", "customer_count")
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   27134317|           306|
|   36063094|            15|
|   49422311|             1|
|   18978114|             1|
|   13455482|             2|
|   32153206|             1|
|   10274872|             4|
|   20977654|             1|
|   48042607|             1|
|   11376183|             7|
|   31762963|             1|
|   11481508|             4|
|   12362676|             2|
|   31176171|            10|
|   44699860|            23|
|   52734389|             8|
|   24303040|             3|
|   26322451|            77|
|   14521576|            30|
|   44269342|            10|
+-----------+--------------+
only showing top 20 rows



In [34]:
# Create the products_table DataFrame and drop duplicates. - product_id, product_title
products_df = df.select(['product_id', 'product_title']).drop_duplicates()
# Also needed porduct_id duplicates since it is a primary field
products_df = products_df.drop_duplicates(subset=['product_id'])
products_df.show(5)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|068986843X|The Stolen Relic ...|
|B000EOTFRQ|  The Grand Alliance|
|B000F83SV0| Lavender Excursions|
|B000F83SWO| Season of the Witch|
|B000F83T2S|   No Enemy But Time|
+----------+--------------------+
only showing top 5 rows



In [23]:
# Create the review_id_table DataFrame. 
# Convert the 'review_date' column to a date datatype with to_date("review_date", 'yyyy-MM-dd').alias("review_date")
review_id_df = raw_review_id_df.select(["review_id", "customer_id", "product_id", "product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_df.show(5)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| RGYFDX8QXKEIR|   33605939|B007KO2MLO|     328837464| 2013-09-09|
|R13CBGTMNV9R8Z|   34058393|B005FLODDE|     764276359| 2013-09-09|
| R7DRFHC0F71O0|   39601147|B00EA3L35O|     535606445| 2013-09-09|
|R27LUKEXU3KBXQ|   17351407|B00BL3JV50|     240053004| 2013-09-09|
|R1VXTPUYMNU687|   10463387|B00CXU7U80|     931529805| 2013-09-09|
+--------------+-----------+----------+--------------+-----------+
only showing top 5 rows



In [24]:
# Create the vine_table. DataFrame - review_id, star_rating, helpful_votes, total_votes, vine, verified_purchase
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "vine", "verified_purchase"])
vine_df .show(5)

+--------------+-----------+-------------+----+-----------------+
|     review_id|star_rating|helpful_votes|vine|verified_purchase|
+--------------+-----------+-------------+----+-----------------+
| RGYFDX8QXKEIR|          4|            0|   N|                N|
|R13CBGTMNV9R8Z|          4|            1|   N|                Y|
| R7DRFHC0F71O0|          5|            0|   N|                N|
|R27LUKEXU3KBXQ|          5|            1|   N|                Y|
|R1VXTPUYMNU687|          5|            1|   N|                N|
+--------------+-----------+-------------+----+-----------------+
only showing top 5 rows



In [25]:
# Store environmental variable
from getpass import getpass
password = getpass('Enter database password :')

# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://amazonbooksreview.cb6a90wdmtjp.us-east-1.rds.amazonaws.com:5432/AwsReview_books"
config = {"user":"postgres", 
          "password": password, 
          "driver":"org.postgresql.Driver"}

Enter database password :··········


In [26]:
# Write review_id_df to table in RDS - !!!
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [35]:
# Write products_df to table in RDS
# about 3 min
products_df.write.jdbc(url=jdbc_url, table='products_table', mode=mode, properties=config)

In [36]:
# Write customers_df to table in RDS
# 5 min 14 s
customers_df.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)

In [37]:
# Write vine_df to table in RDS
# 11 minutes
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)